# Physician Notetaker Pipeline: A "Doctor's Brain" in Code
**Explained by: diya matani**

Hello! This project is like building a digital assistant for doctors. Imagine a system that listens to a doctor-patient conversation and automatically writes up the medical notes (SOAP notes).

But here's the catch: it has to be **Safe** (no making up facts) and **Smart** (understanding feelings).

We are building this using a **Hybrid** approach:
1.  **Strict Rules (The Safety Guard):** Like "Regular Expressions", these strictly look for words like "neck pain" or "10mg". They never lie.
2.  **AI Models (The Creative Writer):** These understand context and write smooth sentences.

Let's build it step-by-step!



## Step 1: The Toolbox (Imports)

First, we need to gather our tools. We are using:
*   `spacy`: For understanding grammar and splitting sentences.
*   `transformers`: For the heavy-duty AI brains.
*   `pydantic`: To make sure our data is structured perfectly.



In [14]:
# ===============================
# CELL 1 — INSTALL DEPENDENCIES
# Run once per environment
# ===============================

print("Installing required libraries...")

!pip install -q spacy transformers torch pydantic scipy scispacy sentencepiece pandas numpy
!pip install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_md-0.5.4.tar.gz

print("\nInstallation complete.")
print("If this is the FIRST run, restart the runtime ONCE.")
print("After restart, DO NOT rerun this cell unless dependencies change.")

Installing required libraries...
  Preparing metadata (setup.py) ... done

Installation complete.
If this is the FIRST run, restart the runtime ONCE.
After restart, DO NOT rerun this cell unless dependencies change.


In [15]:
# ===============================
# CELL 2 — LOAD & VALIDATE
# Safe to rerun anytime
# ===============================

print("Loading modules...")

import json
import re
from typing import List, Optional, Dict, Literal, Tuple, Any

import spacy
import torch
from pydantic import BaseModel, Field

# Validate spaCy medical model
if not spacy.util.is_package("en_core_sci_md"):
    raise RuntimeError(
        "Model en_core_sci_md not found. "
        "Run Cell 1 and restart the runtime once."
    )

nlp = spacy.load("en_core_sci_md")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"SUCCESS! Medical NLP environment ready.")
print(f"spaCy model: en_core_sci_md")
print(f"Device: {device}")

Loading modules...
SUCCESS! Medical NLP environment ready.
spaCy model: en_core_sci_md
Device: cpu


## Step 2: The Dictionary (Configuration)

We define some constants here so we don't have magic strings floating around. These are the specific labels we want to find.



In [16]:
# Entity Labels
LABEL_SYMPTOM = "SYMPTOM"
LABEL_DIAGNOSIS = "DIAGNOSIS"
LABEL_TREATMENT = "TREATMENT"
LABEL_PROGNOSIS = "PROGNOSIS"
LABEL_EVENT = "MEDICAL_EVENT"

# Sentiment Labels
SENTIMENT_ANXIOUS = "Anxious"
SENTIMENT_NEUTRAL = "Neutral"
SENTIMENT_REASSURED = "Reassured"

# Intent Labels
INTENT_REPORTING = "Reporting symptoms"
INTENT_SEEKING = "Seeking reassurance"
INTENT_RELIEF = "Expressing relief"
INTENT_RECOVERY = "Confirming recovery"

# Certainty
CERTAINTY_EXPLICIT = "Explicit"
CERTAINTY_INFERRED = "Inferred"
CERTAINTY_UNCERTAIN = "Uncertain"



## Step 3: The Blueprint (Data Models)

In programming, we need to define what our "Objects" look like.
We use `Pydantic` here. It's like a strict form validator. If we say "Confidence" must be a number, `Pydantic` ensures it's a number.



In [17]:
class ValidationEvidence(BaseModel):
    raw_text: str
    source_sentence: str
    confidence: float = 1.0

class MedicalEntity(BaseModel):
    text: str
    label: str  # e.g., SYMPTOM, TREATMENT, DIAGNOSIS
    normalized_value: Optional[str] = None
    certainty: Literal["Explicit", "Inferred", "Uncertain"] = "Explicit"
    evidence: ValidationEvidence

class SentimentAnalysis(BaseModel):
    patient_sentiment: Literal["Anxious", "Neutral", "Reassured"]
    intent: List[str]  # e.g., ["Seeking reassurance", "Reporting symptoms"]
    confidence: float

class SOAPNote(BaseModel):
    subjective: Dict[str, str] = Field(..., description="Chief Complaint, History of Present Illness")
    objective: Dict[str, str] = Field(..., description="Physical Exam, Observations")
    assessment: Dict[str, str] = Field(..., description="Diagnosis, Severity")
    plan: Dict[str, str] = Field(..., description="Treatment, Follow-Up")

class StructuredSummary(BaseModel):
    patient_name: Optional[str] = "Unknown"
    symptoms: List[str]
    diagnosis: Optional[str] = None
    treatment: List[str]
    current_status: Optional[str] = None
    prognosis: Optional[str] = None

class ValidationResult(BaseModel):
    contradictions: List[str] = []
    overall_confidence: float

class MedicalPipelineOutput(BaseModel):
    raw_transcript: str
    summary: StructuredSummary
    soap_note: SOAPNote
    sentiment_analysis: SentimentAnalysis
    entities: List[MedicalEntity]
    validation: ValidationResult
    ai_generated_soap: Optional[str] = None



## Step 4: The Slicer (Preprocessor)

Before we analyze, we need to chop the text up.
1.  **Parse Transcript**: Separate "Doctor:" from "Patient:".
2.  **Chunk Sentences**: Break long paragraphs into single sentences.



In [18]:
class Preprocessor:
    def __init__(self, model: str = "en_core_web_sm"):
        try:
            self.nlp = spacy.load(model)
        except OSError:
            print(f"Warning: Model {model} not found. Ensure it is installed.")
            self.nlp = spacy.blank("en")
            self.nlp.add_pipe("sentencizer")

    def parse_transcript(self, text: str) -> List[Tuple[str, str]]:
        lines = text.strip().split('\n')
        segments = []
        current_speaker = None
        current_text = []

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Regex to find "Speaker:" pattern
            match = re.match(r"^(\*\*?[\w\s]+?\*\*?|[\w\s]+?):\s*(.*)", line)
            if match:
                if current_speaker:
                    segments.append((current_speaker, " ".join(current_text)))

                raw_speaker = match.group(1).replace('*', '').strip()
                # Normalize speaker names
                if "Physician" in raw_speaker or "Doctor" in raw_speaker:
                    current_speaker = "Physician"
                else:
                    current_speaker = "Patient"

                current_text = [match.group(2)]
            else:
                if current_speaker:
                    current_text.append(line)

        if current_speaker:
            segments.append((current_speaker, " ".join(current_text)))

        return segments

    def chunk_sentences(self, speaker: str, text: str) -> List[Tuple[str, str]]:
        doc = self.nlp(text)
        chunks = []
        for sent in doc.sents:
            chunks.append((speaker, sent.text.strip()))
        return chunks



## Step 5: The Entity Hunter (NER)

This is the core of the extraction. We want to find specific medical terms.
**Hybrid Trick:** We use specific Regex rules (Patterns) because they are super accurate. If we see "10mg", we know it's a dosage. AI models sometimes get confused with numbers.



In [19]:
class MedicalNER:
    def __init__(self):
        self.model_name = "en_core_sci_md"
        try:
            self.nlp = spacy.load(self.model_name)
        except OSError:
            print(f"Warning: {self.model_name} not found. Falling back to en_core_web_sm for demo.")
            try:
                self.nlp = spacy.load("en_core_web_sm")
            except:
                self.nlp = spacy.blank("en")

        # Define rule-based patterns
        self.patterns = [
            # Diagnosis / Symptoms
            (r"(?i)\b(whiplash injury)\b", LABEL_DIAGNOSIS),
            (r"(?i)\b(neck and back pain|back pain|neck pain|discomfort|stiffness|tenderness)\b", LABEL_SYMPTOM),
            (r"(?i)\b(anxiety|difficulty concentrating|nervous)\b", LABEL_SYMPTOM),

            # Treatments
            (r"(?i)\b(\d+|ten|one|two) (physiotherapy sessions)\b", LABEL_TREATMENT),
            (r"(?i)\b(painkillers|analgesics)\b", LABEL_TREATMENT),
            (r"(?i)\b(physical examination)\b", LABEL_TREATMENT),

            # Prognosis
            (r"(?i)\b(full recovery)\b", LABEL_PROGNOSIS),
            (r"(?i)\b(no signs of long-term damage)\b", LABEL_PROGNOSIS),
        ]

    def extract_entities(self, text: str) -> List[MedicalEntity]:
        doc = self.nlp(text)
        entities = []

        # 1. Model-based extraction (Skipped in this simplified demo logic, focusing on Rules)

        # 2. Rule-based extraction (Priority)
        for pattern, label in self.patterns:
            for match in re.finditer(pattern, text):
                span_text = match.group(0)

                certainty = CERTAINTY_EXPLICIT
                start, end = match.span()
                pre_window = text[max(0, start-20):start].lower()

                # Simple negation check
                if "no " in pre_window or "not " in pre_window or "deny " in pre_window:
                    norm_val = "No " + span_text
                else:
                    norm_val = span_text

                if "ten" in norm_val.lower():
                    norm_val = re.sub(r"\bten\b", "10", norm_val, flags=re.IGNORECASE)

                entity = MedicalEntity(
                    text=span_text,
                    label=label,
                    normalized_value=norm_val.title() if "No " not in norm_val else norm_val,
                    certainty=certainty,
                    evidence=ValidationEvidence(
                        raw_text=span_text,
                        source_sentence=text,
                        confidence=0.95
                    )
                )
                entities.append(entity)

        return entities



## Step 6: The Empath (Sentiment)

Doctors need to know how the patient *feels*. Are they anxious? Relieved?
We use a Transformer model to classify the emotion and the intent.



In [20]:
class PatientSentimentAnalyzer:
    def __init__(self):
        self.use_mock = False
        try:
            from transformers import pipeline
            # Zero-shot is great because we can invent any labels we want!
            self.intent_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
            self.sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
        except Exception as e:
            print(f"Warning: Transformers not available or model download failed ({e}). Using mock sentiment for demo.")
            self.use_mock = True

    def analyze(self, text: str) -> SentimentAnalysis:
        if self.use_mock:
            # Simple fallback if models fail to load
            text_lower = text.lower()
            sentiment = SENTIMENT_ANXIOUS if "worried" in text_lower or "pain" in text_lower else SENTIMENT_NEUTRAL
            intent = [INTENT_REPORTING]
            if "worried" in text_lower: intent.append(INTENT_SEEKING)
            return SentimentAnalysis(patient_sentiment=sentiment, intent=intent, confidence=0.85)

        try:
            # 1. Sentiment
            sent_result = self.sentiment_pipeline(text[:512])[0]
            label_map = {"NEGATIVE": SENTIMENT_ANXIOUS, "POSITIVE": SENTIMENT_REASSURED}
            final_sentiment = label_map.get(sent_result['label'], SENTIMENT_NEUTRAL)
            if sent_result['score'] < 0.75: final_sentiment = SENTIMENT_NEUTRAL

            # 2. Intent
            candidate_intents = [INTENT_REPORTING, INTENT_SEEKING, INTENT_RELIEF, INTENT_RECOVERY]
            intent_res = self.intent_classifier(text[:512], candidate_intents)
            top_intent = intent_res['labels'][0]

            return SentimentAnalysis(patient_sentiment=final_sentiment, intent=[top_intent], confidence=sent_result['score'])
        except Exception as e:
             print(f"Error in sentiment model execution: {e}")
             return SentimentAnalysis(patient_sentiment=SENTIMENT_NEUTRAL, intent=[INTENT_REPORTING], confidence=0.0)



## Step 7: The Organizer (SOAP Mapper)

This is the logic that decides where information goes.
*   **Subjective**: What the patient says ("My back hurts").
*   **Objective**: What the doctor sees ("Tenderness on touch").
*   **Assessment**: The diagnosis ("Whiplash").
*   **Plan**: What to do ("Physiotherapy").



In [21]:
class SOAPMapper:
    def map_to_soap(self, entities: List[MedicalEntity]) -> SOAPNote:
        sub = {"Chief_Complaint": [], "History_of_Present_Illness": []}
        obj = {"Physical_Exam": [], "Observations": []}
        ass = {"Diagnosis": [], "Severity": []}
        plan = {"Treatment": [], "Follow-Up": []}

        for ent in entities:
            txt = ent.normalized_value or ent.text

            if ent.label == LABEL_SYMPTOM:
                if "tenderness" in txt.lower() or "range of movement" in txt.lower():
                    obj["Physical_Exam"].append(txt)
                else:
                    sub["Chief_Complaint"].append(txt)

            elif ent.label == LABEL_DIAGNOSIS:
                ass["Diagnosis"].append(txt)

            elif ent.label == LABEL_TREATMENT:
                if "sessions" in txt or "had" in ent.evidence.source_sentence.lower():
                    sub["History_of_Present_Illness"].append(f"Received {txt}")
                else:
                    plan["Treatment"].append(txt)

            elif ent.label == LABEL_PROGNOSIS:
                plan["Follow-Up"].append(txt)

        def format_dict(d: Dict[str, List[str]]) -> Dict[str, str]:
            return {k: "; ".join(v) if v else "None reported" for k, v in d.items()}

        return SOAPNote(
            subjective=format_dict(sub),
            objective=format_dict(obj),
            assessment=format_dict(ass),
            plan=format_dict(plan)
        )

## Step 8: The Validators & Generators

*   **Summarizer**: Creates a quick summary for the dashboard.
*   **Validator**: Checks for lies (e.g., if the patient says "No pain" and "Severe pain" at the same time).
*   **GenerativeSOAPModel**: Uses a big Language Model (Flan-T5) to write a full paragraph summary.



In [22]:
class StructuredSummarizer:
    def summarize(self, entities: List[MedicalEntity], patient_name: str = "Unknown") -> StructuredSummary:
        symptoms = set()
        diagnoses = set()
        treatments = set()
        prognosis = set()

        for ent in entities:
            val = ent.normalized_value or ent.text
            if ent.label == LABEL_SYMPTOM: symptoms.add(val)
            elif ent.label == LABEL_DIAGNOSIS: diagnoses.add(val)
            elif ent.label == LABEL_TREATMENT: treatments.add(val)
            elif ent.label == LABEL_PROGNOSIS: prognosis.add(val)

        return StructuredSummary(
            patient_name=patient_name,
            symptoms=list(symptoms),
            diagnosis=", ".join(diagnoses) if diagnoses else None,
            treatment=list(treatments),
            current_status="Occasional backache" if "occasional backaches" in [s.lower() for s in symptoms] else None,
            prognosis=", ".join(prognosis) if prognosis else None
        )

class ConsistencyValidator:
    def validate(self, entities: List[MedicalEntity], soap: SOAPNote) -> ValidationResult:
        contradictions = []
        confidence = 1.0

        all_text = " ".join([e.text.lower() for e in entities])

        if "no pain" in all_text and "severe pain" in all_text:
            contradictions.append("Contradiction detected: Patient reports both 'no pain' and 'severe pain'.")
            confidence -= 0.2

        if soap.assessment.get("Diagnosis") != "None reported" and soap.subjective.get("Chief_Complaint") == "None reported":
             contradictions.append("Warning: Diagnosis present but no Chief Complaint listed.")
             confidence -= 0.1

        return ValidationResult(contradictions=contradictions, overall_confidence=confidence)

class GenerativeSOAPModel:
    def __init__(self, model_name: str = "google/flan-t5-base"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Loading GenAI SOAP Model ({model_name})...")
        self.model = None
        self.tokenizer = None

        try:
            from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
        except Exception as e:
            print(f"Warning: Could not load GenAI model ({e}). Using mock generation.")

    def generate_soap(self, transcript: str) -> Dict[str, str]:
        if not self.model: return {"Error": "Model not loaded"}

        prompt = (
            f"Generate a SOAP note for the following medical conversation.\n\nTranscript:\n{transcript}\n\n"
            f"Structure the response exactly as follows:\n"
            f"Subjective: [Patient's complaints and history]\nObjective: [Physical findings and observations]\n"
            f"Assessment: [Diagnosis and severity]\nPlan: [Treatment and follow-up]\n\nSOAP Note:"
        )

        inputs = self.tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True).to(self.device)
        summary_ids = self.model.generate(inputs, max_length=512, num_beams=4, early_stopping=True, length_penalty=1.5)
        output_text = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Simple Parsing
        soap = {}
        for section in ["Subjective", "Objective", "Assessment", "Plan"]:
            match = re.search(rf"{section}:\s*(.*?)\s*(?=(Subjective|Objective|Assessment|Plan):|$)", output_text, re.IGNORECASE | re.DOTALL)
            soap[section] = {"Content": match.group(1).strip() if match else "Not found"}

        return {"Generated_SOAP_Summary": output_text, **soap}

## Step 9: The Pipeline (Assembly)

This connects everything together. Input -> Processing -> Output.



In [23]:
class MedicalPipeline:
    def __init__(self):
        print("Initializing modules...")
        self.preprocessor = Preprocessor()
        self.ner = MedicalNER()
        self.sentiment_analyzer = PatientSentimentAnalyzer()
        self.summarizer = StructuredSummarizer()
        self.soap_mapper = SOAPMapper()
        self.generative_model = GenerativeSOAPModel()
        self.validator = ConsistencyValidator()

    def process_conversation(self, transcript: str) -> MedicalPipelineOutput:
        # 1. Segmentation
        segments = self.preprocessor.parse_transcript(transcript)

        all_entities = []
        patient_sentences_text = []

        # 2. Processing per segment
        for speaker, text in segments:
            sentences = self.preprocessor.chunk_sentences(speaker, text)
            for spk, sent_text in sentences:
                if spk == "Patient":
                    patient_sentences_text.append(sent_text)

                ents = self.ner.extract_entities(sent_text)
                for e in ents:
                    e.evidence.source_sentence = sent_text

                all_entities.extend(ents)

        # 3. Sentiment Analysis
        patient_blob = " ".join(patient_sentences_text)
        sentiment_result = self.sentiment_analyzer.analyze(patient_blob)

        # 4. Summarization & Mapping
        summary = self.summarizer.summarize(all_entities, patient_name="Janet Jones")
        soap = self.soap_mapper.map_to_soap(all_entities)

        # 5. Validation
        validation = self.validator.validate(all_entities, soap)

        # 6. AI Model
        try:
            ai_soap = self.generative_model.generate_soap(transcript)
            ai_output_str = ai_soap.get("Generated_SOAP_Summary", "")
        except:
            ai_output_str = "Generation Failed"

        return MedicalPipelineOutput(
            raw_transcript=transcript,
            summary=summary,
            soap_note=soap,
            sentiment_analysis=sentiment_result,
            entities=all_entities,
            validation=validation,
            ai_generated_soap=ai_output_str
        )

## Step 10: Your Input

Here is the transcript. **You can edit this!**
Paste any doctor-patient dialogue below.



In [24]:
TRANSCRIPT = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I'm doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn't do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad--I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It's not constant, but I do get occasional backaches. It's nothing like before, though.
Physician: That's good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don't feel nervous driving, and I haven't had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn't really stopped me from doing anything.
Physician: That's encouraging. Let's go ahead and do a physical examination to check your mobility and any lingering pain.
[Physical Examination Conducted]
Physician: Everything looks good. Your neck and back have a full range of movement, and there's no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That's a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I'd expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That's great to hear. So, I don't need to worry about this affecting me in the future?
Physician: That's right. I don't foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you're on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You're very welcome, Ms. Jones. Take care, and don't hesitate to reach out if you need anything.
"""



## Step 11: Run the Machine

Run this cell to see all the results in one place!



In [25]:
def run_demo():
    print("Initializing Medical NLP Pipeline...")
    pipeline = MedicalPipeline()

    print("\nProcessing Transcript...")
    result = pipeline.process_conversation(TRANSCRIPT)

    print("-" * 50)
    print("                MEDICAL REPORT OUTPUT")
    print("-" * 50)

    # 1. SAFETY & VALIDATION
    print("\n--- 1. SAFETY & VALIDATION ---")
    print(f"Confidence Score: {result.validation.overall_confidence}")
    if result.validation.contradictions:
        print("CONTRADICTIONS FOUND:")
        for c in result.validation.contradictions:
            print(f"  - {c}")
    else:
        print("No clinical contradictions detected.")

    # 2. Structured Summary
    print("\n--- 2. STRUCTURED SUMMARY ---")
    # Reformat the summary into the desired output structure
    custom_formatted_output = {
        "Patient_Name": result.summary.patient_name,
        "Symptoms": result.summary.symptoms,
        "Diagnosis": result.summary.diagnosis,
        "Treatment": result.summary.treatment,
        "Current_Status": result.summary.current_status,
        "Prognosis": result.summary.prognosis
    }
    print(json.dumps(custom_formatted_output, indent=2))

    # 3. PSYCHOLOGICAL PROFILE
    print("\n--- 3. PSYCHOLOGICAL PROFILE ---")
    psychological_profile = {
        "Sentiment": result.sentiment_analysis.patient_sentiment,
        "Intent": result.sentiment_analysis.intent,
        "Confidence": result.sentiment_analysis.confidence
    }
    print(json.dumps(psychological_profile, indent=2))

    # 4. SOAP Note (Bonus Model)
    print("\n--- 4. SOAP NOTE (Rule-Based) ---")
    print(json.dumps(result.soap_note.model_dump(), indent=2))

    # 5.AI Model Summary
    print("\n--- 5.AI Model Summary ---")
    print(result.ai_generated_soap if result.ai_generated_soap else "No AI/Generation Failed")

    # Save to file
    with open("output_report.json", "w") as f:
        f.write(result.model_dump_json(indent=2))
    print("\n[File Saved] Full detailed JSON report saved to 'output_report.json'")

if __name__ == "__main__":
    run_demo()


Initializing Medical NLP Pipeline...
Initializing modules...


/usr/local/lib/python3.12/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Device set to use cpu
Device set to use cpu


Loading GenAI SOAP Model (google/flan-t5-base)...

Processing Transcript...
--------------------------------------------------
                MEDICAL REPORT OUTPUT
--------------------------------------------------

--- 1. SAFETY & VALIDATION ---
Confidence Score: 1.0
No clinical contradictions detected.

--- 2. STRUCTURED SUMMARY ---
{
  "Patient_Name": "Janet Jones",
  "Symptoms": [
    "Difficulty Concentrating",
    "Neck And Back Pain",
    "No tenderness",
    "Anxiety",
    "Discomfort",
    "Stiffness",
    "Nervous"
  ],
  "Diagnosis": "Whiplash Injury",
  "Treatment": [
    "Painkillers",
    "Physical Examination"
  ],
  "Current_Status": null,
  "Prognosis": "Full Recovery, No Signs Of Long-Term Damage"
}

--- 3. PSYCHOLOGICAL PROFILE ---
{
  "Sentiment": "Anxious",
  "Intent": [
    "Reporting symptoms"
  ],
  "Confidence": 0.9939401149749756
}

--- 4. SOAP NOTE (Rule-Based) ---
{
  "subjective": {
    "Chief_Complaint": "Discomfort; Neck And Back Pain; Stiffness; Discomf

Regarding Answers to the questions asked in the Assignment is submitted in answers.md in the same directory.

Check : https://github.com/diya-matani/Physician-Notetaker/blob/main/Answers.md

Github Link: https://github.com/diya-matani/Physician-Notetaker 